In [1]:
import preamble
from virtual_dynamics import SimpleVirtualDynamics
from robot import Robot
from app_loop import AppLoop
from force_guider import ForceGuider
from language_generator import DirectionalLanguageGenerator
from vocalizer import Vocalizer
from realtime_figure import RealtimeFigureWindow
from data_management import TabularDataStore
import numpy as np
import random
from typing import Tuple, List

In [2]:
class HandGuidingGame(AppLoop):

    MASS = 20
    AXES = Robot.TRANSLATION
    PHRASES = [
        [""],
        ["move <direction>"],
        ["move <direction> more"],
        ["You need to move <direction>"]
    ]
    DIRECTION_PAIRS = [("right", "left"), ("forward", "backward"), ("down", "up")]

    def __init__(self, randomize_goal: bool = False, use_guiding_force: bool = True, use_language: bool = True) -> None:
        super().__init__()
        self.use_guiding_force = use_guiding_force
        self.use_language = use_language
        self.randomize_goal = randomize_goal
        self.data_store = TabularDataStore(
            column_names=["t", "F_human", "F_trajectory", "F_error", "F_guide"])

    def startup(self) -> None:
        self.robot = Robot("169.254.9.43",
                           translational_force_deadband=6.0,
                           rotational_force_deadband=0.5)
        self.robot.control.zeroFtSensor()

        self.dynamics = SimpleVirtualDynamics(M=self.MASS, B=50, K=0)

        if self.randomize_goal:
            self.goal_point = np.array(
                [random.uniform(-1.0, -0.5), random.uniform(-0.6, 0.6), random.uniform(0.0, 0.6)])
        else:
            self.goal_point = self.robot.get_pose(self.AXES)

        self.force_guider = ForceGuider(ramp=lambda x: np.sin(np.pi * x) ** 2)
        self.language_generator = DirectionalLanguageGenerator(self.PHRASES, self.DIRECTION_PAIRS, magnitude_interval=12)
        self.vocalizer = Vocalizer(buffer_period=2.0)

    def update(self, t: float, dt: float) -> None:
        period_start = self.robot.control.initPeriod()

        F_human = self.robot.get_force(self.AXES)
        velocity = self.robot.get_velocity(self.AXES)
        delta_x = self.robot.get_pose(self.AXES) - self.goal_point
        F_trajectory = -self.MASS * (2.0 * velocity + 2.0 * delta_x)
        F_trajectory = 0.0 if np.linalg.norm(F_trajectory) == 0.0 else F_trajectory / np.linalg.norm(F_trajectory) * np.linalg.norm(F_human)
        F_error = F_trajectory - F_human

        if self.use_guiding_force and np.linalg.norm(F_error) > 20:
            self.force_guider.initiate_guiding_force(guide_time=2.5)
        
        F_guide = self.force_guider.modulate_force(F_error)

        close_to_goal = np.linalg.norm(delta_x) < 0.05
        not_moving = np.linalg.norm(velocity) < 0.01

        if close_to_goal and not_moving:
            self.stop()

        if self.use_language:
            utterance = self.language_generator.generate(F_error)
            if close_to_goal and not_moving:
                utterance = "You are at the goal. The experiment has completed."
                self.vocalizer.utter(utterance, interupt=True)
            else:
                self.vocalizer.utter(utterance)
        
        self.dynamics.apply_force(F_human + F_guide, dt)
        self.robot.set_velocity(self.dynamics.get_velocity(),
                                Robot.TRANSLATION, acceleration=150)

        self.data_store.append_row([t, F_human, F_trajectory, F_error, F_guide])
        self.robot.control.waitPeriod(period_start)

    def shutdown(self) -> None:
        self.robot.set_velocity(Robot.zeroed_translation_rotation())
    
    def get_data(self) -> TabularDataStore:
        return self.data_store

In [3]:

app = HandGuidingGame(randomize_goal=True, use_guiding_force=True, use_language=True)
app.run_threaded()

figure = RealtimeFigureWindow(subplot_options_set=[{'title': 'Plot', 'colors': ['red', 'green', 'blue']}], )

try:
    while app.is_running():
        t, F_human, F_trajectory, F_error, F_guide = app.get_data().get_all_columns(ordered=True)

        t_current = 1.0 if len(t) == 0 else t[-1]
        for options in figure.subplot_options_set:
            options['xlim'] = (max(t_current - 10, 0), t_current)

        figure.update([(t, F_trajectory)])
finally:
    app.stop()

No matching processes belonging to you were found
